# Proceso ETL para `steam_games`



Este notebook documenta el proceso de Extracción, Transformación y Carga (ETL) aplicado al dataset `steam_games`, fundamental para la preparación de los datos en proyectos de ciencia de datos. El dataset contiene información valiosa sobre juegos disponibles en Steam, incluyendo detalles como el género, nombre, desarrollador, precio, y año de lanzamiento.



Librerías requeridas

In [2]:
import pandas as pd
import re
from utils.utils import cargar_json_gz


## Extracción y Exploración de Datos

Cargamos el dataset y realizamos una exploración inicial para entender su estructura.

In [3]:
# Ruta al archivo steam_games.json.gz
ruta_archivo_steam_games = '../data/steam_games.json.gz'

# Cargamos los datos al DataFrame steam_games
steam = cargar_json_gz(ruta_archivo_steam_games, way=0)

Visualizamos el DataFrame para obtener una primera idea de la estructura de los datos.


In [4]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


## Búsqueda de Nulos 

Identificamos y tratamos valores nulos 

In [5]:
steam.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Eliminamos las columnas 'title', 'early_access', 'url', 'reviews_url', 'tags', 'publisher', 'specs' ya que no se tendra en cuenta en procesamientos posteriores.

In [6]:
steam.drop(['title', 'early_access', 'url', 'reviews_url',
           'tags', 'publisher', 'specs'], axis=1, inplace=True)

Eliminamos las filas con valores nulos para cada variable

In [7]:
steam.dropna(subset=['genres', 'app_name', 'release_date',
             'price', 'id', 'developer'], inplace=True)
steam.reset_index(drop=True, inplace=True)

Revisamos los cambios

In [8]:
print(steam.isnull().sum())
steam

genres          0
app_name        0
release_date    0
price           0
id              0
developer       0
dtype: int64


,genres,app_name,release_date,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...,...
27457,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018-01-04,1.99,745400,Bidoniera Games
27458,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
27459,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
27460,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich


### Transformación de Datos

Aplicamos transformaciones específicas a varias columnas para normalizar y preparar los datos para el análisis.

#### Normalización de Nombres

Normalizamos los nombres en la columna `app_name`, eliminando caracteres especiales y uniformizando el formato de texto.

In [9]:

def limpiar_nombre_aplicacion(nombre):
    """
    Limpia y normaliza los nombres de las aplicaciones.

    Parámetros:
    - nombre (str): El nombre de la aplicación a limpiar.

    Retorna:
    - str: El nombre de la aplicación limpio y normalizado.
    """
    # Eliminamos caracteres especiales excepto espacios y puntos, los cuales son comunes en nombres propios o acrónimos.
    nombre_limpio = re.sub(r'[^\w\s\.]', '', nombre)

    # Normalizamos el espacio: convertimos secuencias de espacios en un único espacio
    nombre_normalizado = re.sub(r'\s+', ' ', nombre_limpio)

    # Aplicamos capitalización por palabras para preservar la importancia de nombres propios y acrónimos.
    return nombre_normalizado.title()


# Aplicamos la función de limpieza a la columna 'app_name'
steam['app_name'] = steam['app_name'].apply(limpiar_nombre_aplicacion)

#### Transformación de Fechas

Extraemos el año de la columna `release_date` y creamos una nueva columna `release_year`.


In [10]:
# buscar los valores que no coinciden con el patrón de fecha yyyy-mm-dd
steam[~steam['release_date'].str.match(r'\d{4}-\d{2}-\d{2}')]

,genres,app_name,release_date,price,id,developer
219,"[Adventure, Indie]",Blueberry Garden,Jun 2009,4.99,29160,Erik Svedäng
459,"[RPG, Strategy]",Age Of Wonders Ii The Wizards Throne,Oct 2010,9.99,61510,Triumph Studios
462,"[RPG, Strategy]",Age Of Wonders Shadow Magic,Oct 2010,9.99,61520,Triumph Studios
463,"[RPG, Strategy]",Age Of Wonders,Oct 2010,5.99,61500,Triumph Studios
546,[Racing],Test Drive Unlimited 2,Feb 2011,19.99,9930,Eden Studios
2404,"[Action, Indie, Racing, Simulation, Sports, Ea...",Altitude0 Lower Faster,Sep 2014,18.99,308080,Gugila
3211,"[Action, Adventure]",Far Cry 4 Valley Of The Yetis,Apr 2015,14.99,324344,"Ubisoft Montreal, Red Storm, Shanghai, Toronto..."
5375,"[Action, Adventure, Casual, Indie, Strategy, E...",Magma Chamber,Apr 2016,4.99,463240,IndieKeep Games
5918,[Utilities],3Dmark Time Spy Benchmark,Jul 2016,Free,496100,Futuremark
9066,"[Indie, Early Access]",Juke,Jul 2017,4.99,665780,Jello Godfri


In [11]:

# encontramos el formato Jun 2009	, 2009-06-01 y unos valores que tienen SOON por lo que se eliminan
con1 = steam['release_date'].str.match(r'\d{4}-\d{2}-\d{2}')
con2 = steam['release_date'].str.match(r'[A-Z][a-z]{2} \d{4}')
con3 = ~con1 & ~con2

steam = steam[con1 & ~con2]
steam.reset_index(drop=True, inplace=True)
steam.isna().sum()

genres          0
app_name        0
release_date    0
price           0
id              0
developer       0
dtype: int64

In [12]:
steam['release_date'] = pd.to_datetime(steam['release_date'])

C:\Users\fcastano5\AppData\Local\Temp\ipykernel_23908\1839756588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam['release_date'] = pd.to_datetime(steam['release_date'])


In [13]:
steam['release_year'] = steam['release_date'].dt.year

C:\Users\fcastano5\AppData\Local\Temp\ipykernel_23908\3739350701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam['release_year'] = steam['release_date'].dt.year


In [14]:
steam

,genres,app_name,release_date,price,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D Poolians,2017-07-24,Free to Play,670290,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,2017
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...
27418,"[Action, Adventure, Casual, Indie]",Kebab It Up,2018-01-04,1.99,745400,Bidoniera Games,2018
27419,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS""",2018
27420,"[Casual, Indie, Strategy]",Logistical South Africa,2018-01-04,4.99,733530,Sacada,2018
27421,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018


#### Ajuste de Precios

Transformamos la columna `price`, reemplazando "Free To Play" por 0.0 y convirtiendo a tipo float.

In [15]:
steam['price'] = steam['price'].apply(
    lambda x: 0.0 if isinstance(x, str) else x).astype('float')

C:\Users\fcastano5\AppData\Local\Temp\ipykernel_23908\1754250626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam['price'] = steam['price'].apply(


#### Explode de Géneros

Manejamos la columna `genres` que contiene listas de géneros, para poder trabajar con ellas de manera más eficiente.

In [16]:
steam = steam.explode("genres").dropna(
    subset=["genres"]).reset_index(drop=True)

## Manejo de duplicados 

Con las transformaciones realizadas, en especial, el explode de la columna generos, facilita el tratamiento de los valores duplicados.

In [17]:
# Conteo de duplicados

steam.duplicated().sum()

1

In [18]:
# revisamos los duplicados
steam[steam.duplicated()]

,genres,app_name,release_date,price,id,developer,release_year
30981,Action,Wolfenstein Ii The New Colossus,2017-10-26,59.99,612880,Machine Games,2017


In [19]:

# Eliminamos los duplicados

steam.drop_duplicates(inplace=True)


### Almacenamiento de Datos con Pickle

Una vez que hemos limpiado y transformado el DataFrame `steam`, procedemos a almacenarlo utilizando el formato Pickle para preservar la integridad de los datos y asegurar un acceso rápido y eficiente para análisis futuros.


In [127]:


ruta_guardar_pickle = "../data/steam_games.pkl"

steam.to_pickle(ruta_guardar_pickle)



,genres,app_name,release_date,price,id,developer,release_year
0,Action,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...
67927,Indie,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018
67928,Racing,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018
67929,Simulation,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018
67930,Casual,Exit 2 Directions,2017-09-02,4.99,658870,"xropi,stev3ns",2017


In [128]:
steam.isna().sum()

genres          0
app_name        0
release_date    0
price           0
id              0
developer       0
release_year    0
dtype: int64

In [130]:
# Verificar que el archivo se haya guardado correctamente

steam_recuperado = pd.read_pickle(ruta_guardar_pickle)
steam_recuperado

,genres,app_name,release_date,price,id,developer,release_year
0,Action,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...
67927,Indie,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018
67928,Racing,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018
67929,Simulation,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,2018
67930,Casual,Exit 2 Directions,2017-09-02,4.99,658870,"xropi,stev3ns",2017




# Reporte final del procesamiento

1. **`app_name`**:
   - **Normalización de Nombres**: Se limpiaron los nombres de las aplicaciones eliminando caracteres especiales y uniformizando el formato de texto. Se permitieron puntos, considerando acrónimos o nombres propios, y se aplicó capitalización por palabras para mantener la importancia de nombres propios y acrónimos.
 
2. **`release_date`**:
   - **Extracción de Año**: Se extrajo el año de la fecha de lanzamiento y se almacenó en una nueva columna `release_year`. Los valores faltantes o incorrectos fueron manejados adecuadamente, excluyendo aquellos registros sin año de lanzamiento válido.


3. **`price`**:
   - **Ajuste de Valores**: Los textos indicativos de juegos gratuitos (como "Free") fueron reemplazados por 0.0, y la columna fue convertida a tipo float para representar correctamente los precios de los juegos.
  .

4. **`genres`**:
   - **Explosión y Limpieza**: La columna, originalmente conteniendo listas de géneros, fue "explotada" para crear filas individuales por cada género, permitiendo una mejor categorización y análisis por género.
  

5. **`id`**:
   - **Renombrado y Limpieza**: La columna `id` fue renombrada a `item_id` para clarificar su propósito como identificador único de contenido. Se aseguró que todos los identificadores fueran únicos y se eliminaron duplicados.
   

6. **Columnas Eliminadas**:
   - Se eliminaron varias columnas (`title`, `early_access`, `url`, `reviews_url`, `tags`, `publisher`, `specs`) que no eran necesarias para los objetivos del análisis, optimizando así el tratamiento de los datos y el rendimiento de las operaciones subsiguientes.


7. **Almacenamiento**:
   - Se guardo el dataframe final usando el formato pickle
